In [1]:
from __future__ import division
from pyomo.environ import *
model = ConcreteModel()
model.x = Var([1,2], domain=NonNegativeReals)
model.OBJ = Objective(expr = 2*model.x[1] + 3*model.x[2])
model.Constraint1 = Constraint(expr = 3*model.x[1] + 4*model.x[2] >= 1)

In [2]:
model

In [1]:
from pyomo.environ import *
from pyomo.opt import SolverFactory
model = AbstractModel()

model.p1 = Param();
model.p2 = Param();
model.p3 = Param();
model.p4 = Param();
model.p5 = Param();
model.p6 = Param();
model.p7 = Param();
model.p8 = Param();
model.p9 = Param();
model.ps = Param();

model.f = Var(initialize = 20, within=PositiveReals)
model.d = Var(initialize = 20, within=PositiveReals)
model.b = Var(initialize = 20, within=PositiveReals)

model.hf = Var(initialize = 20, within=PositiveReals)
model.hd = Var(initialize = 20, within=PositiveReals)
model.hb = Var(initialize = 20, within=PositiveReals)

model.br = Var(initialize=1.5, within=PositiveReals)

model.c = Var(initialize=500000, within=PositiveReals)


def obj_rule(amodel):
    return 10*amodel.hb + amodel.hd + amodel.hf
model.obj = Objective(rule=obj_rule, sense=maximize)

def f_bal_rule(amodel):
    return amodel.f == amodel.p1*amodel.br \
        *(amodel.p2/10.0*amodel.f + amodel.p3*amodel.d) \
        -amodel.hf
model.f_bal = Constraint(rule=f_bal_rule)

def d_bal_rule(amodel):
    return amodel.d == amodel.p4*amodel.d \
        + amodel.p5/2.0*amodel.f - amodel.hd
model.d_bal = Constraint(rule=d_bal_rule)

def b_bal_rule(amodel):
    return amodel.b == amodel.p6*amodel.b \
        + amodel.p5/2.0*amodel.f - amodel.hb
model.b_bal = Constraint(rule=b_bal_rule)

def food_cons_rule(amodel):
    return amodel.c == amodel.p7*amodel.b \
        + amodel.p8*amodel.d + amodel.p9*amodel.f
model.food_cons = Constraint(rule=food_cons_rule)

def supply_rule(amodel):
    return amodel.c <= amodel.ps
model.supply = Constraint(rule=supply_rule)

def birth_rule(amodel):
    return amodel.br == 1.1 + \
        0.8*(amodel.ps - amodel.c)/amodel.ps
model.birth = Constraint(rule=birth_rule)

def minbuck_rule(amodel):
    return amodel.b >= 1.0/5.0*(0.4*amodel.f + amodel.d)
model.minbuck = Constraint(rule=minbuck_rule)

In [2]:
from pyomo.opt import (SolverFactory,
                       TerminationCondition)
from abstract1 import (create_sudoku_model,
                    print_solution,
                    add_integer_cut)

# define the board
board = [(1,1,5),(1,2,3),(1,5,7), \
         (2,1,6),(2,4,1),(2,5,9),(2,6,5), \
         (3,2,9),(3,3,8),(3,8,6), \
         (4,1,8),(4,5,6),(4,9,3), \
         (5,1,4),(5,4,8),(5,6,3),(5,9,1), \
         (6,1,7),(6,5,2),(6,9,6), \
         (7,2,6),(7,7,2),(7,8,8), \
         (8,4,4),(8,5,1),(8,6,9),(8,9,5), \
         (9,5,8),(9,8,7),(9,9,9)]

model = create_sudoku_model(board)

solution_count = 0
while 1:

    with SolverFactory("cplex") as opt:
        results = opt.solve(model)
        if results.solver.termination_condition != \
           TerminationCondition.optimal:
            print("All board solutions have been found")
            break

    solution_count += 1

    add_integer_cut(model)

    print("Solution #%d" % (solution_count))
    print_solution(model)

	Could not locate the 'cplex' executable, which is required for solver cplex


ApplicationError: No executable found for solver 'cplex'

In [3]:
#!/usr/bin/env python2
# -*- coding: utf-8 -*-

import argparse
import os.path
import importlib

from pyomo.environ import *
from pyomo.opt import SolverFactory

def instance_factory(abstract_model_name, path_of_parameters):
    module_of_model = importlib.import_module(abstract_model_name)
    model = module_of_model.model

    data = DataPortal()

    path_of_contacts = os.path.join(path_of_parameters, 'contacts.tab')
    path_of_vertices = os.path.join(path_of_parameters, 'vertices.tab')
    path_of_edges = os.path.join(path_of_parameters, 'edges.tab')
    path_of_boundaries = os.path.join(path_of_parameters, 'boundaries.tab')


    data.load(filename=str(path_of_contacts),
              param=(model.xs_of_contacts, model.ys_of_contacts),
              index=model.contacts)
    data.load(filename=str(path_of_vertices),
              param=(model.xs_of_vertices, model.ys_of_vertices),
              index=model.vertices)
    data.load(filename=str(path_of_edges),
              param=(model.as_of_edges, model.bs_of_edges, model.cs_of_edges),
              index=model.edges)
    data.load(filename=str(path_of_boundaries),
              param=(model.as_of_boundaries, model.bs_of_boundaries, model.cs_of_boundaries),
              index=model.boundaries)

    return model.create_instance(data)

def solve_instance(instance):
    solver = SolverFactory('couenne')

    results = solver.solve(instance, tee=True)
    instance.solutions.load_from(results)

    return instance

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Optimizes the pose of an object inside constraints')
    parser.add_argument('--model_name', '-m', type=str, required=True)
    parser.add_argument('--data', '-d',
                        type=str, required=True,
                        help='The path to the folder containing the two .tab files.')

    args = parser.parse_args()

    instance = instance_factory(args.model_name, args.data)
    results = solve_instance(instance)
    results.display()

usage: ipykernel_launcher.py [-h] --model_name MODEL_NAME --data DATA
ipykernel_launcher.py: error: the following arguments are required: --model_name/-m, --data/-d


SystemExit: 2

/Users/yerin/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2870: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [10]:
from pyomo.opt import SolverFactory
import pyomo.environ
opt = SolverFactory("cplex")
results = opt.solve(model)

	Could not locate the 'cplex' executable, which is required for solver cplex


ApplicationError: No executable found for solver 'cplex'

In [5]:
solver = SolverFactory('cplex.sh')

results = solver.solve(instance, tee=True)

	Failed to create solver with name 'cplex.sh':
	Failed to set executable for solver asl. File with name=cplex.sh either does not exist or it is not executable. To skip this validation, call set_executable with validate=False.


NameError: name 'instance' is not defined

In [12]:
solver = SolverFactory('couenne', executable='./couenne')

	Failed to create solver with name 'couenne':
	Failed to set executable for solver asl. File with name=./couenne either does not exist or it is not executable. To skip this validation, call set_executable with validate=False.


In [8]:
instance = model.create()
instance.y[0] = 1
instance.y[1] = 0

opt = SolverFactory("cplex")

results = opt.solve(instance, warmstart=True)

	Model.create_instance() to create a concrete instance from an abstract
	model.  You do not need to call Model.create() for a concrete model.
	constructed model; returning a clone of the current model instance.


AttributeError: 'ConcreteModel' object has no attribute 'y'

In [6]:
import os

In [8]:
print(os.environ['PATH'])

/Users/yerin/anaconda3/bin:/Users/yerin/anaconda3/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/TeX/texbin
